In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 데이터셋 가져오기

In [2]:
import numpy as np

In [31]:
import pandas as pd
df = pd.read_csv(r'C:\KI\source\01_python\data\ch13_df(결측치중위값으로대체).csv')
df.head()

,지역명,평당분양가격,연도,월
0,서울,18189.0,2013,12
1,부산,8111.0,2013,12
2,대구,8080.0,2013,12
3,인천,10204.0,2013,12
4,광주,6098.0,2013,12


- 지역명2 : 지역명 필드를 label encoding하여 추가
- 독립변수 : 지역명2, 연도, 월
- 타겟변수 : 평당분양가격
- 독립변수와 타겟변수의 scale 조정
    * 정규화(MinMaxScaler) 작업 후 : 지역명2m, 연도m, 월m
    * 표준화(StandardScaler) 작업 후 : 지역명2s, 연도s, 월s
   => 지역명, 연도, 월, 지역명2, 지역명2n, 연도n, 월n, 지역명2s, 연도s, 월s

# 2. 지역명의 label encoding
- 지역명2

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
le = LabelEncoder()
labeling_data = le.fit_transform(df['지역명'])
labeling_data

array([ 8,  7,  5, ...,  3,  2, 14])

In [34]:
df['지역명2'] = labeling_data # 문자 데이터를 가나다순으로 정렬 후 숫자로 변환
df['지역명2']

0        8
1        7
2        5
3       11
4        4
        ..
2171    13
2172    12
2173     3
2174     2
2175    14
Name: 지역명2, Length: 2176, dtype: int32

In [35]:
df.sample(5)

,지역명,평당분양가격,연도,월,지역명2
347,경기,10573.0,2015,8,1
2174,경남,13252.8,2024,8,2
1351,세종,11355.3,2020,8,9
307,부산,9515.0,2015,6,7
1909,광주,18176.4,2023,5,4


# 3. MinMaxScale 조정

In [36]:
X = df[['지역명2', '연도', '월']].values # numpy 배열로 추출한 독립변수 (.values | to_numpy() 사용)
y = df[['평당분양가격']].values # 2차원으로 만들어야 함!
X.shape, y.shape

((2176, 3), (2176, 1))

In [37]:
from sklearn.preprocessing import MinMaxScaler

In [38]:
scaler_X = MinMaxScaler()
df[['지역명2m', '연도m', '월m']] = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler()
df[['평당분양가격m']] = scaler_y.fit_transform(y)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757


In [39]:
# 정규화(MinMaxScaler) 작업 후 : 지역명2n, 연도n, 월n

# 4. StandardScale 조정

In [40]:
# 표준화(StandardScaler) 작업 후 : 지역명2s, 연도s, 월s => 지역명, 연도, 월, 지역명2, 지역명2n, 연도n, 월n, 지역명2s, 연도s, 월s

In [41]:
from sklearn.preprocessing import StandardScaler

In [42]:
scaler_X = StandardScaler()
df[['지역명2s', '연도s', '월s']] = scaler_X.fit_transform(X)
scaler_y = StandardScaler()
df[['평당분양가격s']] = scaler_y.fit_transform(y)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203


# 5. 지역명을 one-hot-encoding

- 강원 0 => 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 (희소행렬)
- 서울 8 => 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 (희소행렬)

In [43]:
df.loc[:16, ['지역명', '지역명2']].sort_values(by='지역명2')

,지역명,지역명2
9,강원,0
7,경기,1
15,경남,2
14,경북,3
4,광주,4
2,대구,5
5,대전,6
1,부산,7
0,서울,8
8,세종,9


In [44]:
loc = df.loc[:16, ['지역명', '지역명2']].sort_values(by='지역명2')['지역명'].tolist()
print(loc) # 원핫인코딩 후 열 이름

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [51]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
# 결과는 numpy 배열, 최대 정수가 16이면 0~16 카데고리로 원핫인코딩, 매개변수에 라벨인코딩된 값
# 결과는 데이터 프레임, 정수가 곧 카테고리가 되어 원핫인코딩, 매개변수에 문자(라벨인코딩이 안 된 값)
temp1 = to_categorical(df['지역명2'])
temp2 = pd.get_dummies(df['지역명']).values
np.all(temp1 == temp2)

True

In [55]:
df[loc] = to_categorical(df['지역명2'])#.astype(np.int16)
pd.options.display.max_columns = 30
df[loc].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   강원      2176 non-null   float32
 1   경기      2176 non-null   float32
 2   경남      2176 non-null   float32
 3   경북      2176 non-null   float32
 4   광주      2176 non-null   float32
 5   대구      2176 non-null   float32
 6   대전      2176 non-null   float32
 7   부산      2176 non-null   float32
 8   서울      2176 non-null   float32
 9   세종      2176 non-null   float32
 10  울산      2176 non-null   float32
 11  인천      2176 non-null   float32
 12  전남      2176 non-null   float32
 13  전북      2176 non-null   float32
 14  제주      2176 non-null   float32
 15  충남      2176 non-null   float32
 16  충북      2176 non-null   float32
dtypes: float32(17)
memory usage: 144.6 KB


In [66]:
df[loc]

,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2172,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2173,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2174,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
df[loc] = df[loc].astype(np.int16)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
# 원핫인코딩 방법2
df[loc] = pd.get_dummies(df['지역명2'])
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
# 원핫인코딩 데이터 지우기
df = df.drop(loc, axis=1)
df.head(1)

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,18189.0,2013,12,8,0.5,0.0,1.0,0.328198,0.0,-1.875367,1.62196,1.168591


In [65]:
# 원핫인코딩 방법3
df = pd.concat([df, pd.get_dummies(df['지역명'])], axis=1)
df.sample(2)

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
506,전남,6494.4,2016,6,12,0.75,0.272727,0.454545,0.023099,0.816497,-0.910030,-0.115370,-1.032592,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
293,광주,7877.0,2015,5,4,0.25,0.181818,0.363636,0.059170,-0.816497,-1.231809,-0.404925,-0.772356,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
